In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt
import re

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [ ]:
lambdas = np.array([9.5,3.5])


In [ ]:
folder_path = rf"../Structures"
project_name = "20251103_LSU_Mean_Free_Path_lt_uniform_grid"
postprocess_results = []
runtime_ps = 25e-12
min_steps_per_lambda = 15
cuts=[2]
h5_bg = None
ref = True
for direction in ["z"]: 
    for dirpath, dirnames, filenames in os.walk(folder_path):
        try:
            for filename in filenames:
                if filename.endswith(".h5") and filename=="n_3.30_ff_0.2237.h5":
                    ff = float(re.search(r'ff_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    n_eff = np.sqrt((1-ff)+ff*(3.3)**2)

                    n_value = float(re.search(r'n_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    for cut in cuts:
                        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
                            continue 
                        if os.path.isfile(os.path.join(dirpath, filename)):
                            file=os.path.join(dirpath, filename)
                            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                                            ,direction=direction, lambda_range=lambdas,
                                                            box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                                           monitors=["flux"], flux_monitor_position=18,cell_size_manual=45,sim_bg=n_eff**2,
                                                           freqs=180, 
                                                           cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                                                           )
                            id0=""
                            sim=structure_1.sim
                            monitor_field = td.FieldMonitor(
                                center=[0.0, 0.0, 0.0],
                                size=[
                                        structure_1.t_slab_x if structure_1.direction == "x" else td.inf,
                                         structure_1.t_slab_y if structure_1.direction == "y" else td.inf,
                                        structure_1.t_slab_z if structure_1.direction == "z" else td.inf
                                    ],
                                fields=["Ex","Ey","Ez"],
                                name="field_monitor",
                                freqs =structure_1.monitor_freqs,
                                )
                            
                            grid =td.GridSpec.uniform(dl=0.1)
                            
                            sim=sim.copy(update={"monitors":list(sim.monitors) + [monitor_field],"grid_spec":grid})
                            folder_desc = rf"H:\phd stuff\tidy3d\data\{project_name}\n_{n_value:.2f}"
                            os.makedirs(folder_desc, exist_ok=True)
                            sim_name=rf"LSU_{Path(filename).stem}_size_{cut}"
                            if False:#os.path.exists(os.path.join(folder_desc, sim_name+".txt")):
                                print("Exist!")
                            else:
                                if ref:
                                    sim0=sim.copy(update={"structures":[]})
                                    id0 =web.upload(sim0, folder_name=project_name,task_name=sim_name+"_0", verbose=True)
                                    web.start(task_id = id0)
                                    web.monitor(id0)
                                    raise Exception

                                id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
                                ids = id0+'\n' + id
                                with open(os.path.join(folder_desc, sim_name+".txt"), "w") as file:
                                    # Write the string to the file
                                    file.write(ids)
                                web.start(task_id = id)
                                web.monitor(id)
        except Exception as e:
            print(f"Error processing {dirpath}: {e}")
        
    

